# Análisis de sueldos con Python

## Data

* [2021.2 - sysarmy - Encuesta de remuneración salarial Argentina
](https://docs.google.com/spreadsheets/d/1-ZRznwS4TK74o90aOiCKS5SiXxUQ2buN1wxZIMHQmzQ/edit#gid=557755710) 
* [2021.2 - sysarmy - Encuesta de remuneración salarial Latam
](https://docs.google.com/spreadsheets/d/1BkBNt1MHVS7DeIlpgmK9l6krtSQ5t_olRhlcyxMeKy0/edit#gid=557755710)

1. Descargar, leer y unir en una sola estructura de datos. ¿Qué estructura elegiste y por qué?

In [ ]:
# En este primer punto se traerán los datos de los dos archivos para trabajar.
# Se ordenarán de tal forma que simplifique el trabajo posterior, así como la normalizacion de sus valores, creando funciones para simplificar el trabajo.


In [ ]:
# A continuación la importación de la encuesta de Latam y la definición de su estructura que será listas de diccionarios.

In [1]:
import csv

list_keys = []
list_values = []
list_dict_info_latam = []                                     # acá se guarda la info de los dict de Latam

with open('2021.2 - sysarmy - Encuesta de remuneración salarial Latam - Latam.tsv', encoding="utf8") as s_latam:
    latam_reader = csv.reader(s_latam, delimiter='\t')
    for index, line in enumerate(latam_reader):
        #if index > 9:
            #print(f'{latam_line}')
        # Líneas 0 a 9 vacías
        # Líneas 10  -> preguntas
        # Líneas 11+ -> respuestas
        if index == 10:
            list_keys = line
            list_keys[0] = 'País'                              # La pregunta Estoy trabajando en se pasa a llamar País
            dict_base = dict()
            for key in list_keys:
                dict_base[key] = ''
            #print(line)
        elif index >= 11:
            list_values = line
            dict_info = dict_base.copy()
            for i, key in enumerate(dict_info.keys()):
                dict_info[key] = list_values[i]
            #print(dict_info)
            dict_info['Ciudad'] = dict_info[dict_info['País']]  # Se agrega una pregunta Ciudad con respuesta variable
            
            for i, key in enumerate(list_keys[1:19]):           # Para borrar las columnas de los países con sus regiones
                dict_info.pop(key)
            #print(dict_info)
            
            list_dict_info_latam.append(dict_info)
        



In [ ]:
# A continuación la importación de la encuesta de Argentina y la definición de la estructura listas de diccionario.

In [2]:
import csv

list_keys = []
list_values = []
list_dict_info_arg = []                           # Acá se guarda la info de los dict de Argentina

with open('2021.2 - sysarmy - Encuesta de remuneración salarial Argentina - Argentina.tsv', encoding="utf8") as s_arg:
    arg_reader = csv.reader(s_arg, delimiter='\t')
    for index, line in enumerate(arg_reader):
        #if index > 8:
            #print(f'{arg_line}')
        # Líneas 0 a 8 vacías
        # Líneas 9   -> preguntas
        # Líneas 10+ -> respuestas
        if index == 9:
            list_keys = line
            list_keys[0] = 'Ciudad'                #La pregunta Dónde estás trabajando se pasa a llamar Ciudad
            dict_base = dict()
            for key in list_keys:
                dict_base[key] = ''
            #print(line)
        elif index >= 10:
            list_values = line
            dict_info = dict_base.copy()
            for i, key in enumerate(dict_info.keys()):
                dict_info[key] = list_values[i]
            #print(dict_info)
            dict_info['País'] = 'Argentina'          # Se agrega una pregunta País con respuesta Argentina
            
            list_dict_info_arg.append(dict_info)
                

In [ ]:
# A continuación seran unidas las dos estructuras en una sola. Argentina se agregará a Latam

In [3]:
list_all = list_dict_info_latam + list_dict_info_arg

In [4]:
print(len(list_all))

6869


In [5]:
print(len(list_dict_info_latam))

429


In [6]:
print(len(list_dict_info_arg))

6440


In [ ]:
#Definiciones de funciones a utilizar

In [8]:
# Función que devuelve el porcentaje

def percentage(total,quantity):
    return round(quantity * 100  / total, 2)


# Función que calcula la paticpación ordenada descendiente de una clave

def participation_per_key_DESC(list_to_analyze, key):
    dict_result_Participacion = {}

    for record in list_to_analyze:
        if record[key] != '':
            if record[key] in dict_result_Participacion:
                # Si la ciudad ya esta en el diccionario resultado incremento el contador
                dict_result_Participacion[record[key]] += 1 
            else:
                # Si la ciudad NO esta en el diccionario resultado inicializo el contador en 1
                dict_result_Participacion[record[key]] = 1   

    return sorted(dict_result_Participacion.items(), key=lambda x: x[1], reverse=True)

# Función que calcula la particpación ordenada descendiente de dos claves

def participation_per_keys_Order_by_Keys(list_to_analyze, key1, key2):
    dict_result_Participacion = {}

    for record in list_to_analyze:
        if record[key1] != '':
            if (record[key1], record[key2]) in dict_result_Participacion:
                # Si la clave ya esta en el diccionario resultado, incremento el contador
                dict_result_Participacion[(record[key1], record[key2])] += 1 
            else:
                # Si la clave NO esta en el diccionario resultado, inicializo el contador en 1
                dict_result_Participacion[(record[key1], record[key2])] = 1   

    return sorted(dict_result_Participacion.items(), key=lambda x: x[0], reverse=False)

# Función para calcular la suma de las cantidades de una lista de tuplas (clave, cantidad)

def participation_count(tuple_list):
    total_count = 0
    for tuple_key_participation in tuple_list:
        total_count += tuple_key_participation[1]
    return total_count
    
# Función normalización Si No No contesta

def normalize_yes_no_empty(value):
    if value == '':
        normalized_value = 'No contesta'
    else:
        normalized_value = value
        
    return normalize_STR(normalized_value)

# Probar funcion normalize_yes_no_empty
#normalize_yes_no_empty('')

# Función para normalizar STR

def normalize_STR(value):
    normalized_value = str(value).strip().capitalize()
    normalized_value = normalized_value.replace('á', 'a')
    normalized_value = normalized_value.replace('é', 'e')
    normalized_value = normalized_value.replace('í', 'i')
    normalized_value = normalized_value.replace('ó', 'o')
    normalized_value = normalized_value.replace('ú', 'u')
    normalized_value = normalized_value.replace('ñ', 'n')
    return normalized_value

# Probar funcion normalize_STR
#normalize_STR('agAáéíóúSgiaASs')

# Funcion para normalizar calores a Float

def normalize_FLOAT(value):
    normalized_value = round(float(value),2)
    return normalized_value

# Probar funcion normalize_FLOAT
#normalize_FLOAT('1.1123123')

# Función para normalizar valores a INT (de STR a INT) 

def normalize_INT(value):
    normalized_value = int(normalize_FLOAT(value))
    return normalized_value

# Probar funcion normalize_INT
#normalize_INT('19.0')


#Función para normalizar la columna Carrera

def normalize_carrera(value):
    normalized_value = str(value).lower()
    if normalized_value in ['.', '-', 'x', 'no', 'cp', 'n/a', 'no estudie', 'ninguna', 'ninguna.', 'ninguna de las anteriores']:
        normalized_value = 'Otros'
    normalized_value = normalized_value.replace('lic.', 'Licenciatura ')
    normalized_value = normalized_value.replace('lic ', 'Licenciatura ')
    normalized_value = normalized_value.replace('tec.', 'Tecnicatura ')
    normalized_value = normalized_value.replace('tec ', 'Tecnicatura ')
    normalized_value = normalized_value.replace('cs.', 'Ciencias ')
    normalized_value = normalized_value.replace('cs ', 'Ciencias ')
    normalized_value = normalized_value.replace('ed.', 'Educacion ')
    normalized_value = normalized_value.replace('ed ' , 'Educacion ')
    normalized_value = normalized_value.replace('rrii', 'Relaciones Internacionales ')
    normalized_value = normalized_value.replace('rrhh', 'Recursos humanos ')
    normalized_value = normalized_value.replace('ing.', 'Ingeniero ')
    normalized_value = normalized_value.replace('ing ', 'Ingeniero ')
    normalized_value = normalized_value.replace('mg.', 'Magister ')
    normalized_value = normalized_value.replace('arq.', 'Arquitecto ')
    normalized_value = normalized_value.replace('arq ', 'Arquitecto ')

    normalized_value = normalize_STR(normalized_value)
    return normalized_value


# Probar funcion normalize_carrera
#normalize_carrera('lic. VhnkYghs')
#normalize_carrera('Arq.')





In [ ]:
# A continuación se normalizarán los elementos 

In [9]:
# Función de normalización:

def normalize_record(dict_record):
    dict_result = {}
    
    for i, key in enumerate(dict_record.keys()):
        if key == 'Ciudad':
            if dict_record[key] == '':
                dict_result[key] = 'SIN CIUDAD'
            else:
                dict_result[key] = dict_record[key]
        elif key == '¿Contribuís a proyectos open source?' or key == '¿Programás como hobbie?':
            dict_result[key] = normalize_yes_no_empty(dict_record[key])
        elif key == 'Años de experiencia':
            #print(dict_record[key])
            dict_result[key] = normalize_FLOAT(dict_record[key])
        elif key in ['Años en la empresa actual', 'Años en el puesto actual']:
            dict_result[key] = normalize_INT(dict_record[key])
        elif key == 'Carrera':
            dict_result[key] = normalize_carrera(dict_record[key])
        elif key == 'Realizaste cursos de especialización' or key == 'Me identifico': 
            dict_result[key] = normalize_STR(dict_record[key])
        else:
            dict_result[key] = dict_record[key]
      
    return dict_result

list_all_normalized = []
for i, key in enumerate(list_all):
    list_all_normalized.append(normalize_record(list_all[i]))
    
#print(list_all_normalized[0].keys())
register_total = len(list_all_normalized)

#Para verificar que se cambió bien el registro:    
print(list_all[0])
print('---------------------------------------------------------------------------------')
print(list_all_normalized[0])
    

{'País': 'Bolivia', 'Tipo de contrato': 'Full-Time', 'Salario mensual o retiro BRUTO (en tu moneda local)': '13500', 'Salario mensual o retiro NETO (en tu moneda local)': '12000', 'Pagos en dólares': '', '¿Cuál fue el último valor de dólar que tomaron?': '', '¿Qué tan conforme estás con tu sueldo?': '3', 'Cómo creés que está tu sueldo con respecto al último semestre': '3', 'Recibís algún tipo de bono': 'Menos de un sueldo', 'A qué está atado el bono': 'Performance de la compañía', '¿Tuviste ajustes por inflación durante 2021?': 'No', '¿De qué % fue el ajuste total?': '0', '¿En qué mes fue el último ajuste?': '0', 'Trabajo de': 'SysAdmin / DevOps / SRE', 'Años de experiencia': '6', 'Años en la empresa actual': '14', 'Años en el puesto actual': '6', '¿Gente a cargo?': '2', 'Plataformas': 'Amazon Web Services, Azure, Docker, Linux, OpenStack', 'Lenguajes de programación o tecnologías.': 'Bash/Shell', 'Frameworks, herramientas y librerías': 'Ninguno de los anteriores', 'Bases de datos': 'A

2. Printear porcentaje de participación por región, ordenado de mayor a menor. Ej: 

```
- Ciudad Autónoma de Buenos Aires - 59.4%
- Formosa - 0.04%
```

In [10]:
sorted_tuples_Participacion_Region = participation_per_key_DESC(list_all_normalized, 'Ciudad')
register_total = participation_count(sorted_tuples_Participacion_Region)

for tuple_record in sorted_tuples_Participacion_Region:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')


 - Ciudad Autónoma de Buenos Aires - 51.71%
 - Provincia de Buenos Aires - 10.1%
 - GBA - 9.2%
 - Córdoba - 7.8%
 - Santa Fe - 5.82%
 - Mendoza - 2.18%
 - Montevideo - 1.7%
 - Entre Ríos - 0.96%
 - Río Negro - 0.76%
 - Tucumán - 0.76%
 - Neuquén - 0.68%
 - Misiones - 0.61%
 - Santiago - 0.58%
 - Chaco - 0.57%
 - México DF - 0.47%
 - Lima - 0.45%
 - Salta - 0.42%
 - San Juan - 0.35%
 - Corrientes - 0.35%
 - Bogotá - 0.28%
 - Chubut - 0.28%
 - Jujuy - 0.28%
 - Medellín - 0.25%
 - Quito - 0.23%
 - La Rioja - 0.23%
 - San Luis - 0.17%
 - Tierra del Fuego - 0.17%
 - Santiago de Cali - 0.13%
 - Ciudad Guatemala - 0.12%
 - La Pampa - 0.12%
 - Santiago del Estero - 0.12%
 - Guadalajara - 0.1%
 - Guayaquil - 0.09%
 - Asunción - 0.09%
 - Santa Cruz de la Sierra - 0.07%
 - Santa Cruz - 0.07%
 - Caracas - 0.07%
 - Querétaro - 0.06%
 - Cochabamba - 0.04%
 - Valparaíso - 0.04%
 - Tunja - 0.04%
 - Manizales - 0.04%
 - San José - 0.04%
 - Loja - 0.04%
 - Monterrey - 0.04%
 - Veracruz - 0.04%
 - Puebla

In [11]:
print(sorted_tuples_Participacion_Region)

[('Ciudad Autónoma de Buenos Aires', 3552), ('Provincia de Buenos Aires', 694), ('GBA', 632), ('Córdoba', 536), ('Santa Fe', 400), ('Mendoza', 150), ('Montevideo', 117), ('Entre Ríos', 66), ('Río Negro', 52), ('Tucumán', 52), ('Neuquén', 47), ('Misiones', 42), ('Santiago', 40), ('Chaco', 39), ('México DF', 32), ('Lima', 31), ('Salta', 29), ('San Juan', 24), ('Corrientes', 24), ('Bogotá', 19), ('Chubut', 19), ('Jujuy', 19), ('Medellín', 17), ('Quito', 16), ('La Rioja', 16), ('San Luis', 12), ('Tierra del Fuego', 12), ('Santiago de Cali', 9), ('Ciudad Guatemala', 8), ('La Pampa', 8), ('Santiago del Estero', 8), ('Guadalajara', 7), ('Guayaquil', 6), ('Asunción', 6), ('Santa Cruz de la Sierra', 5), ('Santa Cruz', 5), ('Caracas', 5), ('Querétaro', 4), ('Cochabamba', 3), ('Valparaíso', 3), ('Tunja', 3), ('Manizales', 3), ('San José', 3), ('Loja', 3), ('Monterrey', 3), ('Veracruz', 3), ('Puebla de Zaragoza', 3), ('Valencia', 3), ('Catamarca', 3), ('La Paz', 2), ('Pereira', 2), ('Heredia', 2),

3. Printear porcentaje de participación por tipo de rol, ordenado de mayor a menor. Ej:  
`- Developer - 39.11 %`

In [12]:
sorted_tuples_Participacion_Rol = participation_per_key_DESC(list_all_normalized, 'Trabajo de')
register_total = participation_count(sorted_tuples_Participacion_Rol)

for tuple_record in sorted_tuples_Participacion_Rol:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')


 - Developer - 40.69%
 - SysAdmin / DevOps / SRE - 11.09%
 - Technical Leader - 6.16%
 - QA / Tester - 4.29%
 - Manager / Director - 4.28%
 - Project Manager - 3.09%
 - HelpDesk - 2.62%
 - BI Analyst / Data Analyst - 2.45%
 - Architect - 2.23%
 - Consultant - 1.92%
 - Data Engineer - 1.76%
 - Business Analyst - 1.69%
 - Networking - 1.54%
 - UX - 1.5%
 - Data Scientist - 1.48%
 - Infosec - 1.43%
 - Recruiter / HR - 1.24%
 - Product Manager - 1.21%
 - DBA - 0.71%
 - Scrum Master - 0.66%
 - Designer - 0.64%
 - VP / C-Level - 0.55%
 - Sales / Pre-Sales - 0.42%
 - Middleware - 0.17%
 - Finance - 0.17%
 - Technical Writer - 0.16%
 - Storage / Backup - 0.12%
 - Analista funcional - 0.09%
 - Docente - 0.07%
 - Product Owner - 0.06%
 - Agile Coach - 0.04%
 - Software Engineer - 0.04%
 - analista funcional - 0.04%
 - Analista Funcional - 0.04%
 - DevOps - 0.04%
 - Coordinador - 0.04%
 - RPA - 0.04%
 - Analista - 0.03%
 - Team leader - 0.03%
 - Support Engineer - 0.03%
 - People Analytics - 0.03

4. Printear porcentaje de respuesta para las preguntas: 
* ¿Contribuís a proyectos Open Source?
* ¿Programás por hobbie?

In [13]:
sorted_tuples_Participacion_Open_Source = participation_per_key_DESC(list_all_normalized, '¿Contribuís a proyectos open source?')
register_total = participation_count(sorted_tuples_Participacion_Open_Source)

print('¿Contribuís a proyectos Open Source?')
for tuple_record in sorted_tuples_Participacion_Open_Source:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')

¿Contribuís a proyectos Open Source?
 - No contesta - 49.44%
 - No - 42.99%
 - Si - 7.57%


In [14]:
sorted_tuples_Participacion_Hobbie = participation_per_key_DESC(list_all_normalized, '¿Programás como hobbie?')
register_total = participation_count(sorted_tuples_Participacion_Hobbie)

print('¿Programás por hobbie?')
for tuple_record in sorted_tuples_Participacion_Hobbie:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')

¿Programás por hobbie?
 - No contesta - 49.44%
 - Si - 28.34%
 - No - 22.22%


5. Printear porcentaje por seniority según años de experiencia, el mapeo es:
```
Junior: de 0 hasta 2 años.
Semi-Senior: de 2 años inclusive hasta 5 años.
Senior: desde 5 años inclusive.
``` 

In [15]:
dict_Seniority_Years = {'Junior': 0, 'Semi-Senior': 0, 'Senior': 0}

for dict_line in list_all_normalized:
    years_of_experiencie = dict_line['Años de experiencia']
    if years_of_experiencie < 2:
        dict_Seniority_Years['Junior'] += 1
    elif years_of_experiencie >= 5:
        dict_Seniority_Years['Senior'] += 1
    else:
        dict_Seniority_Years['Semi-Senior'] += 1
    
#print(dict_Seniority_Years)

register_total = dict_Seniority_Years['Junior'] + dict_Seniority_Years['Senior'] + dict_Seniority_Years['Semi-Senior']

print('Porcentaje de participación por seniority:')
for key in dict_Seniority_Years.keys():
    print(f' {key} - {percentage(register_total,dict_Seniority_Years[key])}%')



Porcentaje de participación por seniority:
 Junior - 15.93%
 Semi-Senior - 26.63%
 Senior - 57.45%


6. Printear porcentaje de personas encuestadas por años en la compañía actual y por años en el puesto actual.

In [16]:
sorted_tuples_Participacion_Anos_Empresa_Actual = participation_per_key_DESC(list_all_normalized, 'Años en la empresa actual')
register_total = participation_count(sorted_tuples_Participacion_Anos_Empresa_Actual)

print('Porcentaje de personas encuestadas por años en la compañia actual:')
for tuple_record in sorted_tuples_Participacion_Anos_Empresa_Actual:
    print(f' - {normalize_INT(tuple_record[0])} años - {percentage(register_total,tuple_record[1])}%')

Porcentaje de personas encuestadas por años en la compañia actual:
 - 0 años - 36.76%
 - 1 años - 16.17%
 - 2 años - 14.62%
 - 3 años - 8.39%
 - 4 años - 4.32%
 - 5 años - 4.11%
 - 6 años - 2.61%
 - 10 años - 2.1%
 - 8 años - 1.98%
 - 7 años - 1.66%
 - 9 años - 1.14%
 - 15 años - 0.92%
 - 11 años - 0.87%
 - 12 años - 0.84%
 - 14 años - 0.61%
 - 13 años - 0.61%
 - 16 años - 0.51%
 - 20 años - 0.48%
 - 18 años - 0.25%
 - 17 años - 0.22%
 - 25 años - 0.17%
 - 21 años - 0.17%
 - 22 años - 0.16%
 - 19 años - 0.06%
 - 23 años - 0.06%
 - 28 años - 0.04%
 - 24 años - 0.04%
 - 26 años - 0.04%
 - 30 años - 0.01%
 - 81 años - 0.01%
 - 2021 años - 0.01%
 - 100 años - 0.01%
 - 32 años - 0.01%
 - 35 años - 0.01%


In [ ]:
#Se detectaron dos valores fuera del rango bla bla bla 

In [17]:
sorted_tuples_Participacion_Anos_Puesto_Actual = participation_per_key_DESC(list_all_normalized, 'Años en el puesto actual')
register_total = participation_count(sorted_tuples_Participacion_Anos_Puesto_Actual)

print('Porcentaje de personas encuestadas por años en su puesto actual:')
for tuple_record in sorted_tuples_Participacion_Anos_Puesto_Actual:
    print(f' - {normalize_INT(tuple_record[0])} años - {percentage(register_total,tuple_record[1])}%')

Porcentaje de personas encuestadas por años en su puesto actual:
 - 0 años - 30.44%
 - 1 años - 19.86%
 - 2 años - 16.68%
 - 3 años - 10.34%
 - 4 años - 5.31%
 - 5 años - 5.15%
 - 6 años - 2.81%
 - 10 años - 2.55%
 - 7 años - 1.53%
 - 8 años - 1.53%
 - 12 años - 0.7%
 - 15 años - 0.7%
 - 9 años - 0.57%
 - 11 años - 0.41%
 - 14 años - 0.33%
 - 13 años - 0.28%
 - 20 años - 0.22%
 - 16 años - 0.13%
 - 18 años - 0.12%
 - 25 años - 0.07%
 - 17 años - 0.07%
 - 21 años - 0.04%
 - 19 años - 0.04%
 - 22 años - 0.03%
 - 28 años - 0.03%
 - 80 años - 0.01%
 - 24 años - 0.01%
 - 33 años - 0.01%
 - 27 años - 0.01%


7. Printear porcentajes de nivel de educación formal y estado, es decir: % educación secundaria, terciaria, universitaria, postgrado, doctorado, postdoctorado, completo, incompleto y en curso para cada uno.

In [18]:
sorted_tuples_Participacion_Nivel_Estudios = participation_per_keys_Order_by_Keys(list_all_normalized, 'Nivel de estudios alcanzado', 'Estado')
#print(sorted_tuples_Participacion_Nivel_Estudios)
register_total_tuples_Participacion_Nivel_Estudios = participation_count(sorted_tuples_Participacion_Nivel_Estudios)
   
print('Porcentaje de nivel de educación formal y su estado:')
for tuple_record in sorted_tuples_Participacion_Nivel_Estudios:
    print(f' - {tuple_record[0][0]} {tuple_record[0][1]} - {percentage(register_total_tuples_Participacion_Nivel_Estudios,tuple_record[1])}%')
    


Porcentaje de nivel de educación formal y su estado:
 - Doctorado Completado - 0.37%
 - Doctorado En curso - 0.29%
 - Doctorado Incompleto - 0.06%
 - Posdoctorado Completado - 0.06%
 - Posdoctorado En curso - 0.03%
 - Posdoctorado Incompleto - 0.03%
 - Posgrado Completado - 4.55%
 - Posgrado En curso - 1.64%
 - Posgrado Incompleto - 0.75%
 - Primario Incompleto - 0.03%
 - Secundario Completado - 5.79%
 - Secundario En curso - 0.14%
 - Secundario Incompleto - 0.46%
 - Terciario Completado - 8.84%
 - Terciario En curso - 4.75%
 - Terciario Incompleto - 3.71%
 - Universitario Completado - 25.19%
 - Universitario En curso - 25.45%
 - Universitario Incompleto - 17.85%


8. Formatear las carreras universitarias:
- Nombres capitalizados
- Reemplazar vocales con tilde por vocales sin tilde.
- Reemplazar `ñ` por `n`
- *lic*, *lic.* por Licenciatura
- *tec*, *tec.* por Tecnicatura
* *cs, *cs.* por Ciencias
* *ed, ed.* por Educación
* Transformaciones que se consideren necesarias

Printear porcentaje según carrera

In [19]:
sorted_tuples_Participacion_Carrera = participation_per_key_DESC(list_all_normalized, 'Carrera')
register_total = participation_count(sorted_tuples_Participacion_Carrera)

print('Porcentaje de participación por carrera:')
for tuple_record in sorted_tuples_Participacion_Carrera:
    print(f' - {(tuple_record[0])} - {percentage(register_total,tuple_record[1])}%')

Porcentaje de participación por carrera:
 - Ingenieria en sistemas de informacion - 19.0%
 - Ingenieria en informatica - 11.34%
 - Analista de sistemas - 8.68%
 - Licenciatura en ciencias de la computacion - 5.66%
 - Licenciatura en sistemas de informacion - 5.66%
 - Licenciatura en informatica - 4.3%
 - Tecnicatura en programacion - 3.39%
 - Ingenieria electronica - 2.99%
 - Ingenieria en computacion - 2.81%
 - Diseno grafico - 2.48%
 - Tecnicatura superior en programacion - 2.36%
 - Licenciatura en administracion - 1.72%
 - Tecnicatura en informatica - 1.57%
 - Ingenieria industrial - 1.18%
 - Licenciatura en analisis de sistemas - 1.18%
 - Analista programador - 1.12%
 - Ingenieria en software - 1.06%
 - Analista programador universitario - 1.0%
 - Ingenieria en telecomunicaciones - 0.97%
 - Analista de computacion - 0.88%
 - Tecnicatura superior en sistemas informaticos - 0.88%
 - Diseno multimedial - 0.85%
 - Tecnicatura en sistemas - 0.82%
 - Tecnicatura superior en analisis de s

9. Printear porcentaje de respuesta a las preguntas:
- ¿Realizaste cursos de especialización?
- Si realizaste, ¿Quién pagó por los cursos de especialización?

In [20]:
dict_Curso_Espec = {'Si': 0, 'No': 0}

for dict_line in list_all_normalized:
    courses = dict_line['Realizaste cursos de especialización']
    if 'Si' in courses:
        dict_Curso_Espec['Si'] += 1
    else:
        dict_Curso_Espec['No'] += 1

register_total = dict_Curso_Espec['Si'] + dict_Curso_Espec['No']

print('Realizaste cursos de especialización:')  
for key in dict_Curso_Espec.keys():
    print(f' {key} - {percentage(register_total,dict_Curso_Espec[key])}%')
    
    


Realizaste cursos de especialización:
 Si - 39.85%
 No - 60.15%


In [21]:
dict_Pago_Curso = {'Empleador': 0, 'Particular': 0, 'A medias': 0}

for dict_line in list_all_normalized:
    courses = dict_line['Realizaste cursos de especialización']
    if 'empleador' in courses and 'particular' in courses:
        dict_Pago_Curso['A medias'] += 1
    elif 'particular' in courses: 
        dict_Pago_Curso['Particular'] += 1
    else: 
        dict_Pago_Curso['Empleador'] += 1

register_total = dict_Pago_Curso['Empleador'] + dict_Pago_Curso['Particular'] + dict_Pago_Curso['A medias']

print('Si realizaste, ¿Quién pagó por los cursos de especialización?')  
for key in dict_Pago_Curso.keys():
    print(f' {key} - {percentage(register_total,dict_Pago_Curso[key])}%')

Si realizaste, ¿Quién pagó por los cursos de especialización?
 Empleador - 64.32%
 Particular - 25.33%
 A medias - 10.35%


10. Printear porcentaje por identidad de género y personas con discapacidad

In [25]:
dict_Genero = {'Hombre': 0, 'Mujer': 0, 'No binario': 0, 'Transgénero': 0, 'Prefiere no decir': 0, 'Fuera de lugar':0 }
# se optó por dejar afuera las respuestas fuera de lugar

for dict_line in list_all_normalized:
    gender = dict_line['Me identifico']
    if 'trans' in gender or 'travesti' in gender or 'transgenero' in gender:
        dict_Genero['Transgénero'] += 1
    elif 'varon' in gender or 'hombre' in gender or 'masculino' in gender or 'macho' in gender:
        dict_Genero['Hombre'] += 1
    elif 'mujer' in gender: 
        dict_Genero['Mujer'] += 1
    elif 'no binario' in gender or 'no binarie' in gender: 
        dict_Genero['No binario'] += 1
    elif 'prefiero no decir' in gender: 
        dict_Genero['Prefiere no decir'] += 1
    else: 
        dict_Genero['Fuera de lugar'] += 1

register_total = dict_Genero['Hombre'] + dict_Genero['Mujer'] + dict_Genero['No binario'] + dict_Genero['Transgénero'] + dict_Genero['Fuera de lugar']

print('Porcentaje de participación por Identidad de Género:')  
for key in dict_Genero.keys():
    print(f' {key} - {percentage(register_total,dict_Genero[key])}%')
    

    

Porcentaje de participación por Identidad de Género:
 Hombre - 0.0%
 Mujer - 0.0%
 No binario - 0.0%
 Transgénero - 0.0%
 Prefiere no decir - 0.0%
 Fuera de lugar - 100.0%


11. Salarios: calcular la mediana salarial para cada una de las siguientes categorías:
- Salario según región
- Salario por rol
- Salario por experiencia
- Salario por nivel de formación
- Salario por carrera
- Salario por tecnología
- Salario por lenguaje de programación
- Salario por género

12. Printear porcentaje y salario según tipos de contrato .

13. En base a los resultados obtenidos confeccionar conclusiones respecto a:

- Rol vs sueldos
- Nivel de educación alcanzada vs sueldos
- Género vs sueldos
- Tecnologías populares

14. Generar gráficos de barra para mostrar los resultados mencionados anteriormente usando strings, ej:

```
developer | -----------------------------------------------
sysadmin  | ---------------------------
QA        | ------------------
          | ..................5%.......10%...............40%
```